<a href="https://colab.research.google.com/github/ezemaudet/TP_NLP1/blob/main/Desafio_2_Custom_embedding_con_Gensim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [1]:
!pip install numpy==1.26.4 pandas==2.2.2 --force-reinstall


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 18.3 MB/s eta 0:00:00
  Attempting uninstall: pytz
    Found existing installation: pytz 2025.2
    Uninstalling pytz-2025.2:
      Successfully uninstalled pytz-2025.2
  Attempting uninstall: tzdata
    Found existing installation: tzdata 2025.2
    Uninstalling tzdata-2025.2:
      Successfully uninstalled tzdata-2025.2
  Attempting uninstall: six
    Found existing installation: six 1.17.0
    Uninstalling six-1.17.0:
      Successfully uninstalled six-1.17.0
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2


In [1]:
!pip install gensim==4.3.3


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 15.7 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:
      Successfully uninstalled scipy-1.15.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset articulos de la bbc.

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("hgultekin/bbcnewsarchive")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/bbcnewsarchive


In [4]:
import os
print("Directorio actual:", os.getcwd())

Directorio actual: /content


In [5]:
os.chdir('/kaggle/input/bbcnewsarchive')

In [6]:
ls

bbc-news-data.csv


In [7]:

df = pd.read_csv("bbc-news-data.csv", sep='\t', engine='python', on_bad_lines='skip')


In [8]:
df.head()

,category,filename,title,content
0,business,001.txt,Ad sales boost Time Warner profit,Quarterly profits at US media giant TimeWarne...
1,business,002.txt,Dollar gains on Greenspan speech,The dollar has hit its highest level against ...
2,business,003.txt,Yukos unit buyer faces loan claim,The owners of embattled Russian oil giant Yuk...
3,business,004.txt,High fuel prices hit BA's profits,British Airways has blamed high fuel prices f...
4,business,005.txt,Pernod takeover talk lifts Domecq,Shares in UK drinks and food firm Allied Dome...


In [9]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('bbc-news-data.csv', sep='/n', header=None)
df.head()

<ipython-input-9-4052f762e430>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('bbc-news-data.csv', sep='/n', header=None)


,0
0,category\tfilename\ttitle\tcontent
1,business\t001.txt\tAd sales boost Time Warner ...
2,business\t002.txt\tDollar gains on Greenspan s...
3,business\t003.txt\tYukos unit buyer faces loan...
4,business\t004.txt\tHigh fuel prices hit BA's p...


In [10]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 2226


### 1 - Preprocesamiento

In [11]:
!pip install tensorflow


In [12]:
#from keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [13]:
# Demos un vistazo
sentence_tokens[:2]

[['category', 'filename', 'title', 'content'],
 ['business',
  '001',
  'txt',
  'ad',
  'sales',
  'boost',
  'time',
  'warner',
  'profit',
  'quarterly',
  'profits',
  'at',
  'us',
  'media',
  'giant',
  'timewarner',
  'jumped',
  '76',
  'to',
  '1',
  '13bn',
  '£600m',
  'for',
  'the',
  'three',
  'months',
  'to',
  'december',
  'from',
  '639m',
  'year',
  'earlier',
  'the',
  'firm',
  'which',
  'is',
  'now',
  'one',
  'of',
  'the',
  'biggest',
  'investors',
  'in',
  'google',
  'benefited',
  'from',
  'sales',
  'of',
  'high',
  'speed',
  'internet',
  'connections',
  'and',
  'higher',
  'advert',
  'sales',
  'timewarner',
  'said',
  'fourth',
  'quarter',
  'sales',
  'rose',
  '2',
  'to',
  '11',
  '1bn',
  'from',
  '10',
  '9bn',
  'its',
  'profits',
  'were',
  'buoyed',
  'by',
  'one',
  'off',
  'gains',
  'which',
  'offset',
  'a',
  'profit',
  'dip',
  'at',
  'warner',
  'bros',
  'and',
  'less',
  'users',
  'for',
  'aol',
  'time',
 

#**Limpieza de los token con funcion de Stop word y Lematizacion**

In [16]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [17]:
sentence_tokens = [
    [lemmatizer.lemmatize(word) for word in sentence if word not in stop_words]
    for sentence in sentence_tokens
]

In [18]:
# Demos un vistazo
sentence_tokens[:2]

[['category', 'filename', 'title', 'content'],
 ['business',
  '001',
  'txt',
  'ad',
  'sale',
  'boost',
  'time',
  'warner',
  'profit',
  'quarterly',
  'profit',
  'u',
  'medium',
  'giant',
  'timewarner',
  'jumped',
  '76',
  '1',
  '13bn',
  '£600m',
  'three',
  'month',
  'december',
  '639m',
  'year',
  'earlier',
  'firm',
  'one',
  'biggest',
  'investor',
  'google',
  'benefited',
  'sale',
  'high',
  'speed',
  'internet',
  'connection',
  'higher',
  'advert',
  'sale',
  'timewarner',
  'said',
  'fourth',
  'quarter',
  'sale',
  'rose',
  '2',
  '11',
  '1bn',
  '10',
  '9bn',
  'profit',
  'buoyed',
  'one',
  'gain',
  'offset',
  'profit',
  'dip',
  'warner',
  'bros',
  'less',
  'user',
  'aol',
  'time',
  'warner',
  'said',
  'friday',
  'owns',
  '8',
  'search',
  'engine',
  'google',
  'internet',
  'business',
  'aol',
  'mixed',
  'fortune',
  'lost',
  '464',
  '000',
  'subscriber',
  'fourth',
  'quarter',
  'profit',
  'lower',
  'precedin

In [19]:
text_to_word_sequence(row[0])

['tech',
 '401',
 'txt',
 'losing',
 'yourself',
 'in',
 'online',
 'gaming',
 'online',
 'role',
 'playing',
 'games',
 'are',
 'time',
 'consuming',
 'but',
 'enthralling',
 'flights',
 'from',
 'reality',
 'but',
 'are',
 'some',
 'people',
 'taking',
 'their',
 'fantasy',
 'lives',
 'too',
 'seriously',
 'when',
 'video',
 'game',
 'world',
 'of',
 'warcraft',
 'hit',
 'the',
 'shops',
 'in',
 'europe',
 'last',
 'week',
 'fans',
 'wrote',
 'in',
 'to',
 'the',
 'bbc',
 'website',
 'to',
 'express',
 'their',
 'delight',
 'and',
 'to',
 'offer',
 'a',
 'warning',
 'an',
 'addiction',
 'to',
 'a',
 'game',
 'like',
 'this',
 'is',
 'far',
 'more',
 'costly',
 'in',
 'time',
 'than',
 'any',
 'substance',
 'could',
 'impair',
 'keep',
 'track',
 'of',
 'time',
 'wrote',
 'travis',
 'anderson',
 'in',
 'texas',
 'some',
 'of',
 'the',
 'comments',
 'were',
 'humorous',
 'this',
 'game',
 'is',
 'so',
 'good',
 "i'm",
 'not',
 'going',
 'to',
 'get',
 'it',
 "there's",
 'no',
 'way',
 

In [20]:
len (sentence_tokens[2225])

1512

### 2 - Crear los vectores (word2vec)

In [21]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [22]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=2,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [23]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [24]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 2226


In [25]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 10277


### 3 - Entrenar embeddings

In [26]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 2873949.75
Loss after epoch 1: 1996569.75
Loss after epoch 2: 1831556.0
Loss after epoch 3: 1799804.5
Loss after epoch 4: 1696790.0
Loss after epoch 5: 1640904.0
Loss after epoch 6: 1653209.0
Loss after epoch 7: 1560654.0
Loss after epoch 8: 1514072.0
Loss after epoch 9: 1409687.0
Loss after epoch 10: 1379364.0
Loss after epoch 11: 1324032.0
Loss after epoch 12: 1347106.0
Loss after epoch 13: 1287674.0
Loss after epoch 14: 1274318.0
Loss after epoch 15: 1250120.0
Loss after epoch 16: 1230566.0
Loss after epoch 17: 1223228.0
Loss after epoch 18: 1227496.0
Loss after epoch 19: 1238690.0


(9323996, 10248180)

### 4 - Ensayar

In [27]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["president"], topn=10)

[('w', 0.48088106513023376),
 ('vice', 0.48035678267478943),
 ('lula', 0.47964778542518616),
 ('vladimir', 0.47583889961242676),
 ('putin', 0.4696238040924072),
 ('silva', 0.4641362130641937),
 ('bush', 0.46359050273895264),
 ("bush's", 0.46316593885421753),
 ("putin's", 0.459325909614563),
 ('cheney', 0.45913705229759216)]

In [46]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["sport"], topn=10)

[('terrorism', 0.04701893776655197),
 ('project', 0.04325863718986511),
 ('successful', 0.03411875665187836),
 ('require', 0.024317657575011253),
 ('cutting', 0.023229850456118584),
 ('alleged', 0.018778787925839424),
 ('throughout', 0.01615065522491932),
 ('short', 0.015258652158081532),
 ('slightly', 0.015196558088064194),
 ('distance', 0.014801381155848503)]

In [42]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["technology"], topn=10)

[('lucent', 0.4531015157699585),
 ('colourful', 0.4349938929080963),
 ('thwart', 0.4131634533405304),
 ('interactivity', 0.4065919816493988),
 ('voip', 0.40312233567237854),
 ('convergence', 0.40176013112068176),
 ('interface', 0.39999791979789734),
 ('boyd', 0.3995105028152466),
 ("toshiba's", 0.3987785577774048),
 ('seamless', 0.396564781665802)]

In [40]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["wife"], topn=10)

[('girlfriend', 0.5791442394256592),
 ('sandra', 0.5398521423339844),
 ('brigitte', 0.5233680009841919),
 ('dee', 0.5211784839630127),
 ('ambani', 0.5189023613929749),
 ('audrey', 0.5173494815826416),
 ('hailey', 0.5105188488960266),
 ('grandmother', 0.5056496858596802),
 ("simpson's", 0.5027886629104614),
 ('mathieu', 0.5001603960990906)]

In [31]:
# Ensayar con una palabra que no está en el vocabulario:
w2v_model.wv.most_similar(negative=["diedaaa"])

KeyError: "Key 'diedaaa' not present in vocabulary"

In [51]:
# el método `get_vector` permite obtener los vectores:
vector_palabra = w2v_model.wv.get_vector("president")
print(vector_palabra)

[ 1.51146859e-01 -1.98227003e-01 -3.27235945e-02 -2.83602029e-01
  1.65008768e-01 -3.23336124e-01  7.09045827e-01 -1.95941582e-01
 -2.54588742e-02 -3.28097105e-01 -1.84344530e-01 -3.77523959e-01
  2.59801093e-02 -4.33530569e-01  6.79491520e-01  2.40927115e-01
 -6.13385141e-01 -1.75934181e-01 -1.34543166e-01 -5.48639715e-01
  1.07306637e-01  4.68712717e-01 -1.56971514e-01 -6.95340931e-01
  7.11166024e-01  9.14732337e-01  2.17476755e-01 -2.91487008e-01
 -6.35632575e-01  2.05437735e-01 -1.28835440e-01 -7.10756302e-01
 -5.88730648e-02 -5.94333827e-01 -7.07837269e-02 -2.75589556e-01
  1.69348523e-01 -1.03636555e-01  7.93183804e-01  3.92447650e-01
 -3.63918841e-01  8.29090655e-01  6.94873214e-01 -1.84715956e-01
  8.98635089e-02 -3.97846878e-01  1.50959983e-01  1.68652326e-01
 -2.57443160e-01 -3.39677125e-01  4.09831583e-01  2.11542696e-02
 -7.79849812e-02 -8.96246374e-01  2.47325569e-01  3.87456864e-01
  1.39205903e-01 -3.70727837e-01  3.80349532e-02  5.22249579e-01
  5.21445945e-02 -1.91532

In [52]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model.wv.most_similar(vector_palabra)

[('president', 1.0),
 ('w', 0.48088106513023376),
 ('vice', 0.48035678267478943),
 ('lula', 0.47964778542518616),
 ('vladimir', 0.47583889961242676),
 ('putin', 0.4696238040924072),
 ('silva', 0.4641362130641937),
 ('bush', 0.46359050273895264),
 ("bush's", 0.46316587924957275),
 ("putin's", 0.459325909614563)]

In [53]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["president"], topn=10)

[('w', 0.48088106513023376),
 ('vice', 0.48035678267478943),
 ('lula', 0.47964778542518616),
 ('vladimir', 0.47583889961242676),
 ('putin', 0.4696238040924072),
 ('silva', 0.4641362130641937),
 ('bush', 0.46359050273895264),
 ("bush's", 0.46316593885421753),
 ("putin's", 0.459325909614563),
 ('cheney', 0.45913705229759216)]

### 5 - Visualizar agrupación de vectores

In [54]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [55]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=100
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

In [56]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="colab") # esto para plotly en colab

### Alumno

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.

#**Conclusiones**

Utilicé un dataset de Kaggle compuesto por 2.226 documentos de noticias de la BBC, distribuidos en 5 categorías temáticas. Para representar el texto numéricamente, implementé una vectorización basada en palabras, cuya calidad mejoró significativamente luego de aplicar técnicas de limpieza al corpus.

Las palabras que son similares tambien se corresponden con los vectores similares. La similitud coseno entre palabras se conserva en el espacio de embeddings, lo que permite identificar relaciones semánticas mediante operaciones vectoriales. Es decir, las palabras con significados similares presentan vectores cercanos entre sí, incluso después de transformaciones como reducción de dimensionalidad o entrenamiento sobre corpus filtrados.

Se ve como en el caso de la palabra "president" lo asocio con el nombre de varios presidentes.
En el plano 2D se ve como government esta cerca de tax y de public.

Primero, eliminé las stopwords del conjunto de tokens, lo cual redujo el número de palabras únicas de 11.552 a 11.388, eliminando términos poco informativos y mejorando la claridad semántica del espacio vectorial.
Posteriormente, apliqué lematización, lo que permitió reducir aún más el vocabulario a 10.277 palabras distintas. Esta etapa resultó clave para agrupar variantes gramaticales bajo una misma raíz, disminuyendo el ruido y mejorando la coherencia entre los vectores generados.

Finalmente, experimenté con el parámetro min_count del modelo Word2Vec. Si bien aumentarlo permite filtrar palabras muy poco frecuentes, observé que valores demasiado altos eliminan términos relevantes, afectando negativamente la calidad de los embeddings. Por lo tanto, concluí que min_count es un parámetro sensible que debe ajustarse cuidadosamente según el tamaño y características del corpus.

Es bastante de mdir la mejora del embeding ya que las pruebas realizadas son meramente cualitativos comparando terminos manualmente.